In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from currency_converter import CurrencyConverter
from currency_converter import RateNotFoundError
from datetime import date

warnings.filterwarnings('ignore')
imdb = pd.read_csv('./data/IMDB_Movies_2000_2020.csv')

In [4]:
imdb = imdb[imdb["budget"].notna()]
imdb = imdb[imdb["usa_gross_income"].notna()]
imdb = imdb[imdb["worlwide_gross_income"].notna()]
imdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3684 entries, 0 to 5475
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_title_id          3684 non-null   object 
 1   title                  3684 non-null   object 
 2   original_title         3684 non-null   object 
 3   year                   3684 non-null   int64  
 4   date_published         3684 non-null   object 
 5   genre                  3684 non-null   object 
 6   duration               3684 non-null   int64  
 7   country                3684 non-null   object 
 8   language_1             3684 non-null   object 
 9   language_2             1574 non-null   object 
 10  language_3             685 non-null    object 
 11  director               3684 non-null   object 
 12  writer                 3683 non-null   object 
 13  actors                 3684 non-null   object 
 14  actors_1               3684 non-null   object 
 15  acto

In [5]:
# convert currency for a certain col
def currency_conv(col_name, df):
    c = CurrencyConverter(decimal=True)
    for ind in df.index:
        if(type(df.at[ind, col_name]) == str):
            if (df.at[ind, col_name].isnumeric() == False):
                temp_str = df.at[ind, col_name]
                #print(temp_str)
                currency = temp_str[0:3]
                #print("currency:", currency)
                amount = int(temp_str[3:].strip())
                try:
                    converted = c.convert(amount, currency.strip(), 'USD', date=date(2022, 3, 31))
                    #print(converted)
                except ValueError:
                    df = df.drop(index=ind)
                except RateNotFoundError:
                    df = df.drop(index=ind)
                else:
                    df.at[ind, col_name] = float(converted)
            
    #df.sort_index()
    return df

In [6]:
imdb = currency_conv("budget", imdb)
imdb = currency_conv("usa_gross_income", imdb)
imdb = currency_conv("worlwide_gross_income", imdb)

In [7]:
print(imdb.columns)

Index(['imdb_title_id', 'title', 'original_title', 'year', 'date_published',
       'genre', 'duration', 'country', 'language_1', 'language_2',
       'language_3', 'director', 'writer', 'actors', 'actors_1', 'actors_f2',
       'description', 'desc35', 'avg_vote', 'votes', 'budget',
       'usa_gross_income', 'worlwide_gross_income', 'reviews_from_users'],
      dtype='object')


In [8]:
imdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3675 entries, 0 to 5475
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_title_id          3675 non-null   object 
 1   title                  3675 non-null   object 
 2   original_title         3675 non-null   object 
 3   year                   3675 non-null   int64  
 4   date_published         3675 non-null   object 
 5   genre                  3675 non-null   object 
 6   duration               3675 non-null   int64  
 7   country                3675 non-null   object 
 8   language_1             3675 non-null   object 
 9   language_2             1568 non-null   object 
 10  language_3             682 non-null    object 
 11  director               3675 non-null   object 
 12  writer                 3674 non-null   object 
 13  actors                 3675 non-null   object 
 14  actors_1               3675 non-null   object 
 15  acto

In [9]:
imdb["budget"] = pd.to_numeric(imdb["budget"])

In [10]:
imdb["usa_gross_income"] = pd.to_numeric(imdb["usa_gross_income"])

In [11]:
imdb["worlwide_gross_income"] = pd.to_numeric(imdb["worlwide_gross_income"])

## Profits for Each Genre

In [12]:
array_genre = []
for str in imdb.genre:
    genres = str.split(',')
    for genre in genres:
        array_genre.append(genre.strip())
        
genre_list, frequency = np.unique(array_genre, return_counts=True)

In [14]:
genre_budget = []
genre_usa_income = []
genre_global_income = []
genre_usa_profit = []
genre_global_profit = []

for genre_name in genre_list:
    sum_budget = imdb["budget"][imdb["genre"].str.contains(genre_name)].sum()
    sum_usa_income = imdb["usa_gross_income"][imdb["genre"].str.contains(genre_name)].sum()
    sum_global_income = imdb["worlwide_gross_income"][imdb["genre"].str.contains(genre_name)].sum()
    sum_usa_profit = sum_usa_income - sum_budget
    sum_global_profit = sum_global_income - sum_budget
    
    genre_budget.append(sum_budget)
    genre_usa_income.append(sum_usa_income)
    genre_global_income.append(sum_global_income)
    genre_usa_profit.append(sum_usa_profit)
    genre_global_profit.append(sum_global_profit)
    
df_genre = {"genre": genre_list,
            "num": frequency,
            "all_budget": genre_budget,
            "avg_budget": genre_budget/frequency,
            "all_usa_income": genre_global_income,
            "avg_usa_income": genre_global_income/frequency,
            "all_global_income": genre_global_income,
            "avg_global_income": genre_global_income/frequency,
            "all_usa_profit": genre_usa_profit,
            "avg_usa_profit": genre_usa_profit/frequency,
            "all_global_profit": genre_global_profit,
            "avg_global_profit": genre_global_profit/frequency
            }
df_genre = pd.DataFrame(df_genre)

In [15]:
df_genre

,genre,num,all_budget,avg_budget,all_usa_income,avg_usa_income,all_global_income,avg_global_income,all_usa_profit,avg_usa_profit,all_global_profit,avg_global_profit
0,Action,1052,7.229487e+10,6.872136e+07,209480432812,1.991259e+08,209480432812,1.991259e+08,8.510990e+09,8.090294e+06,1.371856e+11,1.304045e+08
1,Adventure,782,7.162981e+10,9.159823e+07,240432699372,3.074587e+08,240432699372,3.074587e+08,1.682170e+10,2.151112e+07,1.688029e+11,2.158605e+08
2,Animation,228,1.926918e+10,8.451395e+07,70516176338,3.092815e+08,70516176338,3.092815e+08,7.340647e+09,3.219582e+07,5.124700e+10,2.247675e+08
3,Biography,303,8.006258e+09,2.642330e+07,20750333642,6.848295e+07,20750333642,6.848295e+07,1.728507e+09,5.704642e+06,1.274408e+10,4.205965e+07
4,Comedy,1395,5.222024e+10,3.743386e+07,155103005998,1.111850e+08,155103005998,1.111850e+08,2.043392e+10,1.464797e+07,1.028828e+11,7.375109e+07
5,Crime,684,2.085451e+10,3.048906e+07,46151947427,6.747361e+07,46151947427,6.747361e+07,1.491699e+09,2.180847e+06,2.529743e+10,3.698455e+07
6,Drama,1936,5.302305e+10,2.738794e+07,138467806142,7.152263e+07,138467806142,7.152263e+07,8.586289e+09,4.435067e+06,8.544475e+10,4.413469e+07
7,Family,227,1.515377e+10,6.675668e+07,46611967857,2.053391e+08,46611967857,2.053391e+08,4.474093e+09,1.970966e+07,3.145820e+10,1.385824e+08
8,Fantasy,329,2.151455e+10,6.539376e+07,66551536395,2.022843e+08,66551536395,2.022843e+08,3.516603e+09,1.068876e+07,4.503699e+10,1.368905e+08
9,History,126,4.403455e+09,3.494806e+07,9392674795,7.454504e+07,9392674795,7.454504e+07,-2.715949e+08,-2.155515e+06,4.989220e+09,3.959698e+07


## Profits For Each Country

In [16]:
array_country = []
for str in imdb.country:
    countrys = str.split(',')
    for country in countrys:
        array_country.append(country.strip())
        
country_list, frequency = np.unique(array_country, return_counts=True)

In [17]:
country_budget = []
country_usa_income = []
country_global_income = []
country_usa_profit = []
country_global_profit = []

for country_name in country_list:
    sum_budget = imdb["budget"][imdb["country"].str.contains(country_name)].sum()
    sum_usa_income = imdb["usa_gross_income"][imdb["country"].str.contains(country_name)].sum()
    sum_global_income = imdb["worlwide_gross_income"][imdb["country"].str.contains(country_name)].sum()
    sum_usa_profit = sum_usa_income - sum_budget
    sum_global_profit = sum_global_income - sum_budget
    
    country_budget.append(sum_budget)
    country_usa_income.append(sum_usa_income)
    country_global_income.append(sum_global_income)
    country_usa_profit.append(sum_usa_profit)
    country_global_profit.append(sum_global_profit)
    
df_country = {"country": country_list, 
              "num": frequency,
              "all_budget": country_budget,
              "avg_budget": country_budget/frequency,
              "all_usa_income": country_global_income,
              "avg_usa_income": country_global_income/frequency,
              "all_global_income": country_global_income,
              "avg_global_income": country_global_income/frequency,
              "all_usa_profit": country_usa_profit,
              "avg_usa_profit": country_usa_profit/frequency,
              "all_global_profit": country_global_profit,
              "avg_global_profit": country_global_profit/frequency
              }
df_country = pd.DataFrame(df_country)

In [18]:
df_country

,country,num,all_budget,avg_budget,all_usa_income,avg_usa_income,all_global_income,avg_global_income,all_usa_profit,avg_usa_profit,all_global_profit,avg_global_profit
0,Afghanistan,1,2.000000e+07,2.000000e+07,75011029,7.501103e+07,75011029,7.501103e+07,-4.199922e+06,-4.199922e+06,5.501103e+07,5.501103e+07
1,Algeria,1,1.609645e+07,1.609645e+07,22963701,2.296370e+07,22963701,2.296370e+07,-1.577575e+07,-1.577575e+07,6.867251e+06,6.867251e+06
2,Angola,1,5.000000e+05,5.000000e+05,101729,1.017290e+05,101729,1.017290e+05,-3.996300e+05,-3.996300e+05,-3.982710e+05,-3.982710e+05
3,Argentina,17,1.622324e+08,9.543082e+06,391307270,2.301807e+07,391307270,2.301807e+07,-8.159453e+07,-4.799678e+06,2.290749e+08,1.347499e+07
4,Australia,125,7.064687e+09,5.651750e+07,19705530424,1.576442e+08,19705530424,1.576442e+08,6.385102e+08,5.108081e+06,1.264084e+10,1.011267e+08
...,...,...,...,...,...,...,...,...,...,...,...,...
83,USA,3150,1.457051e+11,4.625559e+07,437428911220,1.388663e+08,437428911220,1.388663e+08,4.255919e+10,1.351085e+07,2.917238e+11,9.261073e+07
84,Ukraine,1,3.000000e+07,3.000000e+07,108979549,1.089795e+08,108979549,1.089795e+08,1.715062e+06,1.715062e+06,7.897955e+07,7.897955e+07
85,United Arab Emirates,28,1.625700e+09,5.806071e+07,4843282289,1.729744e+08,4843282289,1.729744e+08,5.575554e+07,1.991269e+06,3.217582e+09,1.149137e+08
86,Venezuela,1,1.400000e+06,1.400000e+06,3217176,3.217176e+06,3217176,3.217176e+06,-7.075100e+04,-7.075100e+04,1.817176e+06,1.817176e+06


## Profits For Each Actor

In [28]:
# split actors' name
def split_names(col):
    array = []
    imdb_sub = imdb.dropna(subset = [col])
    for ind in imdb_sub.index:
        names = imdb_sub.at[ind, col].split(',')
        for name in names:
            array.append(name.strip())
            
    return np.unique(array, return_counts=True)

In [29]:
actor_list, actor_freq = split_names('actors')
actor_list

array(['50 Cent', 'A. Cheron Hall', 'A. Delon Ellis Jr.', ...,
       'Ørjan Gamst', 'Þorleifur Einarsson', 'Þorsteinn Gunnar Bjarnason'],
      dtype='<U37')

In [30]:
actor_budget = []
actor_usa_income = []
actor_global_income = []
actor_usa_profit = []
actor_global_profit = []

for actor_name in actor_list:
    sum_budget = imdb["budget"][imdb["actors"].str.contains(actor_name)].sum()
    sum_usa_income = imdb["usa_gross_income"][imdb["actors"].str.contains(actor_name)].sum()
    sum_global_income = imdb["worlwide_gross_income"][imdb["actors"].str.contains(actor_name)].sum()
    sum_usa_profit = sum_usa_income - sum_budget
    sum_global_profit = sum_global_income - sum_budget
    
    actor_budget.append(sum_budget)
    actor_usa_income.append(sum_usa_income)
    actor_global_income.append(sum_global_income)
    actor_usa_profit.append(sum_usa_profit)
    actor_global_profit.append(sum_global_profit)
    
df_actor = {"actor": actor_list,
            "num": actor_freq,
            "all_budget": actor_budget,
            "avg_budget": actor_budget/actor_freq,
            "all_usa_income": actor_global_income,
            "avg_usa_income": actor_global_income/actor_freq,
            "all_global_income": actor_global_income,
            "avg_global_income": actor_global_income/actor_freq,
            "all_usa_profit": actor_usa_profit,
            "avg_usa_profit": actor_usa_profit/actor_freq,
            "all_global_profit": actor_global_profit,
            "avg_global_profit": actor_global_profit/actor_freq
            }
df_actor = pd.DataFrame(df_actor)

In [31]:
df_actor

,actor,num,all_budget,avg_budget,all_usa_income,avg_usa_income,all_global_income,avg_global_income,all_usa_profit,avg_usa_profit,all_global_profit,avg_global_profit
0,50 Cent,7,2.270000e+08,3.242857e+07,439019372,6.271705e+07,439019372,6.271705e+07,-3.319207e+07,-4.741724e+06,2.120194e+08,3.028848e+07
1,A. Cheron Hall,1,2.500000e+07,2.500000e+07,30893885,3.089388e+07,30893885,3.089388e+07,-5.471398e+06,-5.471398e+06,5.893885e+06,5.893885e+06
2,A. Delon Ellis Jr.,1,3.200000e+07,3.200000e+07,44102389,4.410239e+07,44102389,4.410239e+07,8.222729e+06,8.222729e+06,1.210239e+07,1.210239e+07
3,A. Jay Radcliff,1,6.000000e+07,6.000000e+07,204594016,2.045940e+08,204594016,2.045940e+08,2.057401e+07,2.057401e+07,1.445940e+08,1.445940e+08
4,A. Russell Andrews,2,7.300000e+07,3.650000e+07,81591954,4.079598e+07,81591954,4.079598e+07,-1.873638e+07,-9.368190e+06,8.591954e+06,4.295977e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
27347,Óscar Lara,1,3.774340e+06,3.774340e+06,78638987,7.863899e+07,78638987,7.863899e+07,3.386944e+06,3.386944e+06,7.486465e+07,7.486465e+07
27348,Óscar Zafra,1,5.600000e+06,5.600000e+06,18853164,1.885316e+07,18853164,1.885316e+07,-5.572234e+06,-5.572234e+06,1.325316e+07,1.325316e+07
27349,Ørjan Gamst,2,4.800978e+06,2.400489e+06,3354274,1.677137e+06,3354274,1.677137e+06,-4.716763e+06,-2.358382e+06,-1.446704e+06,-7.233521e+05
27350,Þorleifur Einarsson,1,1.942675e+06,1.942675e+06,1826583,1.826583e+06,1826583,1.826583e+06,-1.793425e+06,-1.793425e+06,-1.160920e+05,-1.160920e+05


In [34]:
df_actor = df_actor.drop(df_actor[df_actor.num < 3].index)
df_actor

,actor,num,all_budget,avg_budget,all_usa_income,avg_usa_income,all_global_income,avg_global_income,all_usa_profit,avg_usa_profit,all_global_profit,avg_global_profit
0,50 Cent,7,2.270000e+08,3.242857e+07,439019372,6.271705e+07,439019372,6.271705e+07,-3.319207e+07,-4.741724e+06,2.120194e+08,3.028848e+07
5,A.C. Peterson,5,2.080000e+08,4.160000e+07,204800090,4.096002e+07,204800090,4.096002e+07,-1.140146e+08,-2.280293e+07,-3.199910e+06,-6.399820e+05
14,AJ Bowen,5,6.750000e+06,1.350000e+06,28202493,5.640499e+06,28202493,5.640499e+06,1.215820e+07,2.431639e+06,2.145249e+07,4.290499e+06
21,Aamir Khan,8,5.917283e+07,7.396604e+06,474140146,5.926752e+07,474140146,5.926752e+07,-2.990473e+07,-3.738092e+06,4.149673e+08,5.187091e+07
39,Aaron Eckhart,23,1.075950e+09,4.678043e+07,2619989122,1.139126e+08,2619989122,1.139126e+08,2.200234e+08,9.566233e+06,1.544039e+09,6.713214e+07
...,...,...,...,...,...,...,...,...,...,...,...,...
27310,Zoë Bell,3,2.310000e+08,7.700000e+07,467353170,1.557844e+08,467353170,1.557844e+08,-6.273745e+07,-2.091248e+07,2.363532e+08,7.878439e+07
27312,Zoë Kravitz,15,1.276000e+09,8.506667e+07,3308124357,2.205416e+08,3308124357,2.205416e+08,7.212042e+07,4.808028e+06,2.032124e+09,1.354750e+08
27319,Zuleikha Robinson,3,1.395000e+08,4.650000e+07,149955125,4.998504e+07,149955125,4.998504e+07,-5.486172e+07,-1.828724e+07,1.045512e+07,3.485042e+06
27343,Ólafur Darri Ólafsson,3,3.600000e+08,1.200000e+08,872439794,2.908133e+08,872439794,2.908133e+08,-1.317048e+08,-4.390161e+07,5.124398e+08,1.708133e+08


In [24]:
df_genre.to_csv('df_genre.csv')

In [25]:
df_country.to_csv('df_country.csv')

In [35]:
df_actor.to_csv('df_actor.csv')